In [1]:
# Make ophyd listen to pyepics.
import nslsii
import ophyd.signal

import matplotlib.pyplot as plt

ophyd.signal.EpicsSignal.set_defaults(connection_timeout=5)
# See docstring for nslsii.configure_base() for more details
# this command takes away much of the boilerplate for setting up a profile
# (such as setting up best effort callbacks etc)
# nslsii.configure_base(get_ipython().user_ns, 'xpd', pbar=True, bec=True,
#                       magics=True, mpl=True, epics_context=False)


# At the end of every run, verify that files were saved and
# print a confirmation message.
from bluesky.callbacks.broker import verify_files_saved, post_run
# RE.subscribe(post_run(verify_files_saved, db), 'stop')
from bluesky import RunEngine
from bluesky.plans import count

# Uncomment the following lines to turn on verbose messages for
# debugging.
# import logging
# ophyd.logger.setLevel(logging.DEBUG)
# logging.basicConfig(level=logging.DEBUG)

# nslsii.configure_base(get_ipython().user_ns, "xpd",
#                       publish_documents_with_kafka=True)

nslsii.configure_base(get_ipython().user_ns, "xpd-ldrd20-31",
                      publish_documents_with_kafka=True)

RE.md['facility'] = 'NSLS-II'
RE.md['group'] = 'XPD'
RE.md['beamline_id'] = '28-ID-2'

import subprocess


from ophyd import Device, EpicsMotor, EpicsSignal, EpicsSignalRO
from ophyd import Component as Cpt
import time
import pandas as pd
import numpy as np
from ophyd.sim import det4, noisy_det, motor  # simulated detector, motor
import h5py
from datetime import datetime

import bluesky.plan_stubs as bps
from bluesky.plans import count

import matplotlib.pyplot as plt
import glob

import bluesky.plan_stubs as bps
from bluesky.plans import count, scan
from bluesky.utils import Msg


# Make plots update live while scans run.
# from bluesky.utils import install_kicker
# install_kicker()
#%matplotlib notebook
#from bluesky.utils import install_nb_kicker
#install_nb_kicker()


# ## Prepare Data Storage
# from databroker import Broker
# db = Broker.named('xpd-ldrd20-31')

# from databroker import catalog
# db = catalog['xpd']


# ## Add a Progress Bar
# from bluesky.utils import ProgressBarManager
# RE.waiting_hook = ProgressBarManager()

def show_env():
    # this is not guaranteed to work as you can start IPython without hacking
    # the path via activate
    proc = subprocess.Popen(["conda", "list"], stdout=subprocess.PIPE)
    out, err = proc.communicate()
    a = out.decode('utf-8')
    b = a.split('\n')
    print(b[0].split('/')[-1][:-1])


environment variable BLUESKY_LOG_FILE is not set, using default log file path '/nsls2/users/clin1/.cache/bluesky/log/bluesky.log'
environment variable BLUESKY_IPYTHON_LOG_FILE is not set, using default file path '/nsls2/users/clin1/.cache/bluesky/log/bluesky_ipython.log'


Activating auto-logging. Current session state plus future input saved.
Filename       : /nsls2/users/clin1/.cache/bluesky/log/bluesky_ipython.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Exception reporting mode: Minimal


In [2]:
#%matplotlib widget
#%matplotlib inline
#%matplotlib notebook

In [2]:
RE(count([det4], num=1))



Transient Scan ID: 114     Time: 2022-11-17 14:16:11
Persistent Unique Scan ID: '1d515c35-a3ee-4e4e-8b3e-83526b7fdd4b'
New stream: 'primary'
+-----------+------------+------------+
|   seq_num |       time |       det4 |
+-----------+------------+------------+
|         1 | 14:16:11.1 |      1.000 |
+-----------+------------+------------+
generator count ['1d515c35'] (scan num: 114)





('1d515c35-a3ee-4e4e-8b3e-83526b7fdd4b',)

In [9]:
def reset_pumps(pump_list, clear=True, update = '.2 second'):
    for pump in pump_list:
        pump.initialize_pump(clear=clear, update = update)
        pump.infuse_rate_unit.put('ul/min', wait=True)
        pump.infuse_rate.put(100, wait=True)
        pump.withdraw_rate_unit.put('ul/min', wait=True)
        pump.withdraw_rate.put(100, wait=True)
        pump.target_vol_unit.put('ml', wait=True)
        pump.target_vol.put(20, wait=True)


def reset_pumps2(pump_list, clear=True, update = '.2 second'):
    for pump in pump_list:
        pump.initialize_pump(clear=clear, update = update)
        yield from bps.mv(pump.infuse_rate_unit, 'ul/min', 
                          pump.infuse_rate, 100, 
                          pump.withdraw_rate_unit, 'ul/min', 
                          pump.withdraw_rate, 100, 
                          pump.target_vol_unit, 'ml', 
                          pump.target_vol, 20)


def show_pump_status(syringe_list, pump_list, precursor_list, wait=False):
    for input_size, pump, precursor in zip(syringe_list, pump_list, precursor_list):
        print('Name: ' + f'{pump.name}')
        print('Precursor: ' + f'{precursor}')
        pump.check_pump_condition(input_size, wait=wait)
        print('\n')
        

def set_group_vol(syringe_list, pump_list, target_vol_list=['50 ml', '50 ml']):
    for i, j, l in zip(pump_list, target_vol_list, syringe_list):
        vol = float(j.split(' ')[0])
        vol_unit = j.split(' ')[1]      
        yield from i.set_target_vol(l, target_vol = vol, target_unit = vol_unit)

        
        
def set_group_infuse(syringe_list, pump_list, infuse_rate_list = ['100 ul/min', '100 ul/min']):
    for i, k, l in zip(pump_list, infuse_rate_list, syringe_list):
        rate = float(k.split(' ')[0])
        rate_unit = k.split(' ')[1]        
        yield from i.set_infuse(l, infuse_rate = rate, infuse_unit = rate_unit)

        

def set_group_withdraw(syringe_list, pump_list, withdraw_rate_list = ['100 ul/min', '100 ul/min']):
    for i, k, l in zip(pump_list, withdraw_rate_list, syringe_list):
        rate = float(k.split(' ')[0])
        rate_unit = k.split(' ')[1]        
        yield from i.set_withdraw(l, withdraw_rate = rate, withdraw_unit = rate_unit)



def start_group_infuse(pump_list):
    for pump in pump_list:
        yield from pump.infuse_pump2()
        

def start_group_withdraw(pump_list):
    for pump in pump_list:
        yield from pump.withdraw_pump2()
        

def stop_group(pump_list):
    for pump in pump_list:
        yield from pump.stop_pump2()
        


def _readable_time(unix_time):
    from datetime import datetime
    dt = datetime.fromtimestamp(unix_time)
    print(f'{dt.year}{dt.month:02d}{dt.day:02d},{dt.hour:02d}{dt.minute:02d}{dt.second:02d}')
    return (f'{dt.year}{dt.month:02d}{dt.day:02d}'), (f'{dt.hour:02d}{dt.minute:02d}{dt.second:02d}')


def time_in_mixer(rate, rate_unit, length_mm, diameter_mm=1.016):
    const1 = vol_unit_converter(v0=rate_unit[:2], v1='ul')/t_unit_converter(t0=rate_unit[3:], t1='min')
    rate = rate*const1
    mixer_volume_mm3 = np.pi*((diameter_mm/2)**2)*length_mm
    t_sec = 60*mixer_volume_mm3/rate
    return t_sec


def insitu_test(abs_repeat, cor_repeat, csv_path=None, sample='rhodamine', pump_list=None, precursor_list=None, mixer=None, note=None):
    # yield from bps.sleep(2)
    for i in range(abs_repeat):
        yield from qepro.take_uvvis_save_csv(sample_type=sample, csv_path=csv_path, 
                                              spectrum_type='Absorbtion', correction_type='Reference', 
                                              pump_list=pump_list, precursor_list=precursor_list, mixer=mixer, note=note)
    # yield from bps.sleep(2)    
    for j in range(cor_repeat):
        yield from qepro.take_uvvis_save_csv(sample_type=sample, csv_path=csv_path, 
                                              spectrum_type='Corrected Sample', correction_type='Dark', 
                                              pump_list=pump_list, precursor_list=precursor_list, mixer=mixer, note=note)


# def insitu_test2(abs_repeat, cor_repeat, csv_path=None, sample='rhodamine', pump_list=None, precursor_list=None):
#     for i in range(abs_repeat):
#         yield from qepro.take_uvvis_save_csv2(sample_type=sample, csv_path=csv_path, 
#                                               spectrum_type='Absorbtion', correction_type='Reference', 
#                                               pump_list=pump_list, precursor_list=precursor_list)
        
#     for j in range(cor_repeat):
#         yield from qepro.take_uvvis_save_csv2(sample_type=sample, csv_path=csv_path, 
#                                               spectrum_type='Corrected Sample', correction_type='Dark', 
#                                               pump_list=pump_list, precursor_list=precursor_list)

In [ ]:
### Set Pumps, syringes, precursors, mixers info ###

## Available pump list : [dds1_p1, dds1_p2, dds2_p1, dds2_p2, ultra1, ultra2]
# pump_list = [dds1_p1, dds1_p2]

pump_list = ['dds1_p1', 'dds1_p2']
syringe_list = [50, 50]
target_vol_list = ['30 ml', '30 ml']
infuse_rates = [['50 ul/min', '50 ul/min'],
                ['100 ul/min', '100 ul/min'],
                ['150 ul/min', '150 ul/min'],
                ['250 ul/min', '250 ul/min'],
                ['350 ul/min', '350 ul/min'],
                ['500 ul/min', '500 ul/min'],
                ['1000 ul/min', '1000 ul/min']]
precursor_list = ['CsPb 6 mM', 'TetraBr 12 mM']
mixer = ['FEP_30cm']


In [ ]:
### Set UVVIS parameters: sample name, Dark, Bkg, integration time, num of spectra to average ###
sample_name = ['CsPbBr_100uLmin', 'CsPbBr_200uLmin', 'CsPbBr_300uLmin', 
               'CsPbBr_500uLmin', 'CsPbBr_700uLmin', 'CsPbBr_1000uLmin', 
               'CsPbBr_2000uLmin']
int_time = 300 # in ms
num_ave = 9

RE(qepro.take_ref_bkg2(integration_time=int_time, num_spectra_to_average=num_ave))

In [ ]:
### Reset pumps and set target volumes ###
RE(reset_pumps2(pump_list))
RE(set_group_vol(syringe_list, pump_list, target_vol_list=target_vol_list)

In [ ]:
### Start pumps with insitu UVVIS ###

def infuse_pumps_insitu UVVIS(infuse_rate_list, sample_name, pump_list, equilibirum_time, abs_repeat, cor_repeat, precursor_list, mixer, note='CsPbBr_reaction_time'):
    for rate, name in zip(infuse_rates, sample_name):
        yield from set_group_infuse(syringe_list, pump_list, infuse_rate_list = rate)
        yield from start_group_infuse(pump_list)
        yield from bps.sleep(equilibirum_time)
        yield from insitu_test(abs_repeat, cor_repeat, sample=name, pump_list=None, precursor_list=precursor_list, mixer=mixer, note=note)

        

In [6]:
import pandas as pd
import numpy as np

def vol_unit_converter(v0 = 'ul', v1 = 'ml'):
    vol_unit = ['pl', 'nl', 'ul', 'ml']
    vol_frame = pd.DataFrame(data={'pl': np.geomspace(1, 1E9, num=4), 'nl': np.geomspace(1E-3, 1E6, num=4),
                                   'ul': np.geomspace(1E-6, 1E3, num=4), 'ml': np.geomspace(1E-9, 1, num=4)}, index=vol_unit)
    return vol_frame.loc[v0, v1]


def t_unit_converter(t0 = 'min', t1 = 'min'):
    t_unit = ['sec', 'min', 'hr']
    t_frame = pd.DataFrame(data={'sec': np.geomspace(1, 3600, num=3), 
                                 'min': np.geomspace(1/60, 60, num=3), 
                                 'hr' : np.geomspace(1/3600, 1, num=3)}, index=t_unit)
    return t_frame.loc[t0, t1]


def time_in_mixer(rate, rate_unit, length_mm, diameter_mm=1.016):
    const1 = vol_unit_converter(v0=rate_unit[:2], v1='ul')/t_unit_converter(t0=rate_unit[3:], t1='min')
    rate = rate*const1
    mixer_volume_mm3 = np.pi*((diameter_mm/2)**2)*length_mm
    t_sec = 60*mixer_volume_mm3/rate
    return t_sec

In [8]:
time_in_mixer(200, 'ul/min', 300)

72.96587699003967